##### Project Diary

29May2018

Time to get this thing organized.  

So far:

1. built and submitted holdout predictions using only numerical data.  Model accuracy is 0.  That model scored 1.33

1. construct pipeline to do preprocessing.

    1. select numerical data.  Impute values for nans
    2. select text fields.  Merge and transform using count_vectorizer
    3. join numerical and BOW vectors for prediction
    4. predict using one_vs_all(logistic regression)

1. built and submitted holdout predictions using all data (see above steps).  Numerical nans are replaced with default imputation (mean).  Text fields are merged within rows and fed to count_vectorizer (using defaults).  Model scored 0.746 log_loss.  Model accuracy:  0.347.

1. substituted random forest classifier and predict.  Submitted and scored 2.1101.  Model accuracy: 0.905

At this point, I began to wonder about the scoring/log_loss function.  The RFC was pretty accurate but had poor log loss.  Why is that?  I believe the short version is that log_loss penalizes wrong and confident predictions severely.  I did a bunch of messing around with the provided implementation, but I've been unable to come up with something that matches the posted scores.

There are (at least) 3 issues:
1. I don't have the holdout set true labels to compute the loss.
2. I don't know if my prediction/loss on the test set tracks the holdout set.
3. They say they normalize the results across class.  This may be the one piece I haven't done.

Here are a few of my comments:

> I think this is pretty close to what they scored for this model... Nope, that was 1.33 not 1.64. That could be okay, since the training data and holdout set may be from slightly different distributions.
> The key thing to keep in mind is that to measure fit, I need to calculate loss on training set and compare to loss on test set. If it's much better on training, the model is overfit.
> Still think this is the best candidate loss fn so far (i.e. `np.mean(compute_log_loss(probas, truth)*9)`. 

> Next step is to see what comes back from this fn with the next model.

Another source of confusion: it's possible to predict directly on labels rather than one-hot encoding them and treating each as a separate binary classification.  It would be feasable to not one-hot the outputs and predict with most classifiers.  Then the classifier would presumably realize that the labels in each output are mutually exclusive (and complete) and that would take care of the normalizing (i.e. that would just happen).  Some of the writeups on log loss are couched with this assumption.  Another name for log_loss is cross entropy loss.

The immediate problem here is that I've cluttered up the root notebook with all this feverish fiddling with the loss function.

What needs to happen now before it goes any further is to break the loss experiments out into another file.  In order to efficiently work on that it may be necessary to save results that I don't have.

Here's what I need out of a model:
1. prediction on holdout set for submission (I have this already saved to files).

This stuff isn't yet saved to files:

1. to calculate training loss
    1. yhat on training set
    1. y from training set 
1. to calculate test loss
    1. yhat on test test
    1. y from test set
    
If I capture those model outputs and true labels, I can play with the loss function to see if I can get close to what they're scoring.  

Important: the reason this matters is that if I need to tune (by grid search or other method) I will need a loss function that behaves like theirs so I can optimize.



So, right now, the task is to checkpoint, make a copy and rip out all the loss stuff and stick with the program of following the template, with the addition of adding the pieces I need to work on the loss stuff separately.

Course notebook is now cleaned up, error-free, top to bottom.  

So steam through the course and see where we end up.  As far as the loss goes, I'm close and I can come back when I've gotten as much oomph as I can


####  30 and 31 May2018

The course notebook is complete.  In addition to the coursework, I submitted models presented in chapters 3 and 4.  These models have more feature engineering, but perform worse than the 2nd model that does minimal preprocessing.  Some get fairly close but they all take __*much*__ longer to fit.

One option for summarizing the model performance would be to graph accuracy and log_loss for all of these models.  

First needed action is build local log_loss metric that tracks the contest metric.  I need this to be able to tell if model changes are helpful or not.

For a first step just build the first model (numeric variables) and grind out a log loss function that works correctly.

#### 01/02 June

* Discovered 'official' driven data log_loss metric

    Doesn't match mine or scores on submitted data.
    

* Stripped out tutorial stuff and created first_models.ipynb for AJ call.

    Also, created and commited driven *subdirectory* below *springboard*.


##### To do

* Write test/train yhats, ys to files so I can run the metrics without running the models (too slow).  Come up with a way to organize that.


#### AJ call 

AJ suggests that the work so far should be written up as the milestone report.  Include EDA, data story and any other pieces required in the curriclum.  Classification report can maybe add a lot of information for interpretation (log_loss is a) problematic and b) non-trivial to interpret.

I think the CR would be helpful. What I need to find out is 'is it applicable'.  I'm wondering also about a heatmap for the confusion matrix.  Can that be applied?

He suggested I might ignore the contest and investigate what I have so far.  I think it's perhaps informative to submit but it doesn't make sense to care about the result as long as the metric doesn't make sense.

Document modeling results.  It's fine to have questions.

To-be-done section can describe what other approaches make sense in alloted time frame.










#### 03/04 June

I've done a bunch of work looking at metrics but I feel like I'm lost in the woods.

I checked the 'official' metric on the first 2 models.  It tracks submission on mod0 but not on mod1.

There's a email back and forth with AJ about what needs to be in a milestone report.  He's approved a project plan - but I'm not sure I understand what it is.

Built a small metrics displayer that takes y, yhat, yhat_proba and shows many metrics.

##### Don't forget
* build uniform probabilies
    1. should score 2.0455 if submitted.
    2. see what my metrics score it.
    
* need an EDA
    * immediate need is to see target and features columns

#### 05 June

I've figured out a bit about multi-multi land.  Lots of clfs will do it; you don't have to do what DD did.  But maybe they did it that way is so the output comes out in the submission format.  Clf change probability out format depending on whether you hand them multiple target columns or not.

So that's some learning, but I'm still confused.

AJ kind of wanted to know what the classifier is doing the way we all usually do with accuracy, precision/recall/F1.  Me, I like plotting the confusion matrix.

And I want to know that too because we're changing what's going on - the scoring of submissions gives me no feel for how well the classifiers work.

To underline that, yesterday I read some commentary by the 3 top finishers in the contest.  One guy said the holdout set is sufficiently distinct that he couldn't get any feedback except by submitting.  None of these guys' models were overfit.  

It seemed that they all did a bunch of work just to make the classifiers less sure about wrong predictions.  That's a big win on log_loss.

One way to know more about the classifiers shown in the tutorials is to predict targets along with probabilities and look at classification report and confusion matrix.

I'm not trusting their metric really, because I'm not sure it's fair to predict 104 columns independently and then normalize the sum across a one-hot target group.

It seems better to me to actually do the multiclass classification for each target.  That should ensure that probabilities for all labels on a row sum to one.  Seems like doing one-vs-rest on single target is different than one-vs-rest on all the one-hot target.  Don't know if it's true.


#### 06/07 June

I need to clean up in a big way.  Here's what I think should happen.

1. consolidate repos/driven and repos/springboard/driven - we don't need two copies
1. make local directories
    1. make a submissions directory.  document what's what.
    1. make data directory
    1. make directory for model outputs (nd.arrays)
    
* It occurs to me that I could just do the classification over the unencoded targets.  Then, the proba output would be a list of n_targets, with array of probas for each target.  That would be easy to transform to required output.

        This doesn't work with logistic regression.  Logreg is happy to predict multiple labels for a __*single*__ target (using one-vs-rest internally), but won't do multiple targets.  Random Forest (and other classifiers) will do this.  See http://scikit-learn.org/stable/modules/multiclass.html#multiclass.

        For logistic regression, another approach is to make a classifier for each target.  That allows the use of standard sklearn metrics (classification report, etc.) as well as sklearn.metric.log_loss.  

* There's another thing to try here: multinomial logistic regression which I assume actually means multilabel.  Is it faster?  Is its output in the same format?

    It doesn't seem to work very well.  It needs special treatment and I was unable to get it to converge.
    


#### 08jun2018


#### Cleaning
1. Copy/move everything to the right place; get rid of crud.
2. Commit the necessary to github.

#### What's left to do in mod0_multiple_ways

* I want to compare accuracy, classification report and confusion matrix for all 9 targets.  Are the 2 approaches getting similar results on these metrics?  If not, why??

* I need to figure out how to print a big confusion matrix.


##### write model prediction outputs to files

That way I can have a sterile field to write format converters
 
These are for predictions.  Do I need corresponding fns for probabilities?  I don't think so, because the 1st models format (104 columns of probas) can just be split by column ranges.  The 2nd models (9 arrays) can just be merged, like in this file.

* __*flat_to_labels*__ takes 104 columns of one-hot encoded labels, outputs 9 columns of string labels
* __*labels_to_flat*__ takes 9 columns of string labels outputs 104 columns of one-hot encoded labels


    
These fns if run one after the other should produce original input.

#### 09-11 jun2018

Saturday call with AJ.  See emails.

##### A thought: I should submit m0_ml prediction.  It is more accurate.  my metric (and theirs) is not worse than mod0.  Curious about where they score it.


#### Worked on flat_to_labels

I think I understand what's different about the first model and my implementation of the first model.  

The model from the tutorial is using 0.5 as a probality threshold to predict a 1 in a one-hot encoded variable.  It computes independent probabilities for each column.  So its predictions are not what one expects.  It has nothing contraining it to choose the most likely label for a target.  Sometimes (often) it predicts no label for a target because none of the constituent columns has probability > 0.5.  It *could* predict multiple labels within a target.  

My implementation uses the fact that logistic regression can do the right thing when presented a multilabel problem.  I think (though I should prove this to myself) that it does one-vs-rest internally but enforces the constraint that sum of label probabilities must be one and when it predicts, it chooses the highest probability label.

It may be a feature of log loss that it doesn't care (although the drivendata metric has the somewhat artificial characteristic  that it normalizes probabilities within targets) about this enabling the perhaps computationally more efficient 1vs103 methodology.  I say this because my implementation has the same log loss as theirs, even though mine is much more effective at prediction (but still bad).

#### 12jun2018

__*flat_to_labels*__ seems to be working.  Need to test on mod0 probabilities.

#### 23jun2018

Flat_to_labels is working; see the notebook standard_metrics.ipynb.  Built a plot of accuracy profile for each of the presented classifiers.  

Next step is to enumerate the changes between classifiers because they skip steps and it's hard to tell how much difference is attributable to which change.

One way to fix this would be to make one change at a time and measure performance before and after.

#### 24jun2018

AJ suggests that I maybe want to keep the goal in mind and think about the product (maybe a good way to do that is to do the milestone report tomorrow).

What is the finished product?

* Problem description
* Data description/EDA
* Performance analysis of models
    * I want to incrementalize.  We'll see if scaling doesn't speed it all up (though it won't help the last one)...
    * Was there one in between my mod0 and mod1, that just did text features?  No, but maybe there should be in the incremental version.
    * Describe approach (i.e. one_vs_all plus logisticRegression).  Advantages/disadvantages.  Output formats for predictions and probabilities for different classifiers
   
* Describe improvements attributable to feature engineering
* Improving the model is a nice-to-have

#### week of 25jun2018 mon

On the way with the incremental approach:

* mod0 - numerical features only
    * the first presented model
    
    
* mod0_0 - same as mod0, but use standard scaler before prediction
    *  Model fits much faster; no impact on predictions
    
    
* mod0_1 - same as mod0 but use standard scaler and default imputer before prediction
    *  Fits slightly faster; slightly worse classification performance


* mod1 - pipeline, numerical features and text features (fillna with empty string; combine all text columns within row; default count vectorizer)
    * the second model from the tutorial
    

* __mod1_1__ - same as above but ignore numerical data
    * removing the numerical data is a big win with average accuracy and F1 scores of  0.974.  Log loss is 0.0940.  This is good enough for top 10 finish in competition.  Note: count vectorizer version runs into problems with OneVsAll(LogReg(), n_jobs=-1).  Work around is to set n_jobs=1.  Works but quite slow. Winning classification quality.


* __mod_1_1_1__: another workaround: replace all numerical features with 0s.  This allows n_jobs=-1. Also allows pipeline to work almost completely as written.  It works well and uses all processors yielding the same results as the slower, 1-processor version above. Fits in 464sec instead of 827sec.


* mod1_2:  Swapping in HashingVectorizer allows n_jobs=-1 (full parallel operation); much faster.  Classification performance slightly degraded.  Using parameters __*non_negative=True, norm=None, binary=False*__  gives equivalent classification performance to CountVectorizer (worse by insignificant but detectible amount), but is 100% slower than defaults.


* mod1_3: use 1 character tokens.  Same as mod1_2 but allows 1 character alphanumeric tokens (tutorial claims this is better).  Performance is slightly degraded from default token pattern (min 2-character alphanumeric)


* __mod3_1__: same as mod1_1_1, but adds n_gram_range=(1, 2).  Best model so far. 
    __log loss on training set: 0.0467, log loss on test set: 0.0573__


* mod3_1a: same as mod3_1, adds MaxAbsScaler before classification.  Slightly faster, slightly worse classification..


* mod3_2: same as mod3_1, but selects 300 best features (SelectKBest(chi2, 300)). Significant worse performance (unacceptable), somewhat faster to fit.

* Mod3_2a: same as mod3_1, but selects 3000 best features (SelectKBest(chi2, 3000)). Less performance degradation, minor fitting speed-up.

* Mod3_3: same as mod3_1, but selects 15000 best features (SelectFromModel(RandomForest...)). Less performance degradation, minor fitting speed-up.

#### 26jun2018 tue

##### Question: are single character tokens useful?  It would seem that the only way to find out is to run the vectorizer with that pattern instead of the default.  Answer: no.  Classification quality is reduced.

##### The next step after mod1 is mod2.  Mod2 substitutes RandomForest for OneVsAll(LogReg()).  Tutorial immediately leaves RF behind after mod2 and returns to  OneVsAll(LogReg()) adding more feature engineering.  My approach is to ignore RF for now and continue with the feature modification steps.  When that is complete, we can return  and apply RF on all the engineered feature sets.

##### So in part_2, we start with mod_1_1_1 (best performing so far not including RF) and add the following:

* text features
    1. in count_vectorizer: ngram_range=(1, 2)
        * improves classification; doubles time
    2. SelectKBest(chi2, chi_k) 
        * degrades classification; speeds up fitting (but not dramatically)
* scale with MaxAbsScaler
        * very little impact
        
Of these only ngram_range=(1, 2) is helpful.

#### 27jun2018 wed

See above


#### 28jun2018 thu

Time to change directions a bit.  I think I should do as much of a milestone report as I can before Saturday.

Here's what I wrote above with some re-org/additions:

* Problem description
* Data description/EDA
* Modeling approach
    * multi-target-multi-class
        * tutorial approach
        * other approaches
            * Random Forest
            * 1 multilabel classifier for each target
* Feature engineering
* Performance analysis of models
* To do
    * modeling
    * feature engineering
        * text processing
        * numerical features
        * dimension reduction
      
I want to incrementalize. We'll see if scaling doesn't speed it all up (though it won't help the last one)...
Was there one in between my mod0 and mod1, that just did text features? No, but maybe there should be in the incremental version.
Describe approach (i.e. one_vs_all plus logisticRegression). Advantages/disadvantages. Output formats for predictions and probabilities for different classifiers
Describe improvements attributable to feature engineering

Improving the model is a nice-to-have


#### sat

AJ meeting.  What I have in the draft milestone is good, but it's too long (though everything there is usable for final report).
The last page is really more in the spirit of a milestone report.  This is what the curriculum says about the report.

> * Define the problem
> * Identify your client
> * Describe your data set, and how you cleaned/wrangled it
> * List other potential data sets you could use
> * Explain your initial findings
> * Share the Capstone Project 1 code and milestone report related to Github repository

Item 4. There aren't other relevant data sets.
Item 6. I think this means publish the code on Github (and provide a readme for ease of consumption)

##### Note: remember, I want to use RandomForest on the best feature engineering combination I can find.  Another way to do this problem might be to do native RF(multilabel) on nine targets.   All that's needed is one copy of the features (14 word vecs) that can be pre-computed.  Labels can be left alone.  

##### Note: one of the things I want to do at the end is error analysis.  There are at least 2 benefits: 1) possibly can use the information gleaned to tweak feature engineering. 2) If most errors are localizable (certain targets/labels), the model can be delivered with appropriate warnings along with its performance characteristics.

#### more saturday

My workflow is heavily impacted by the choice to include multiple models in the same notebook.  When there's a crash (too often, Redmond), the easiest thing to do is re-run the notebook to be able to continue work.  Many of the models are long running and it seems that it would be better to have one model per file. Then once a model has run and reported, there should be no need to run again.

Another approach would be to store the models.  Water-cooler gossip suggests that it may not be workable.  Documentation is straightforward.

Right now, what I want to do is look at the tokens from each feature column in train and test sets and in the holdout set, but I can't work on that because I wanted to quickly run another model (3_2b, SelectKBest, 15k features) and, oops - I started executing that nb.  Now I need to let it run to completion (2 hours?).  Never again.

#### sun 01jul

Did some on EDA.  This is supposed to be part of milestone report.  The EDA has:

1. targets - distribution in the  dataset
2. numerical features - ranges, problems noted
3. text features - number of tokens in each and in total for dataset and holdout

Also looked at 2nd place finishers code.  His first file is readable and interpretable but he's a lot more patient than I am.  Has 1 million features and I wasn't able to fit even one model (out of 104) in the time I was will to sit there (maybe 15 min).

Revisited 3rd place - it's done in R so useless to me.  He says he coded unique texts as numbers.

Look again at 1st place model.  I understand what he's doing a lot better (though the actual SGD implementation is a bit obscure without further scrutiny).  

The hashing he's doing is just a little code wrapped around hash (actually reducing the hash space to a known constant).

In the parsing he just appends every hash generated to a list.   That list ends up being the features for the X.  It's sparse by construction.

The online approach updates the all the model weights for each X[]

The weights are a list 2^18 long (matching the hash space).  There's one of these for each (104) model.  There's another piece exactly as big to hold the gradients from before.

He's doing these to generate the features:
* hashing the original whole text in a column  
* hashing the individual token in the text
* hashing individual token plus column index, encodind occurence of  word in feature
* hashing certain whole feature texts in certain pairs (and one triple) of features to capture interactions between columns.
* binning the numerical features

He says it requires 2.7 and pypy, but I'm doing it in my setup now (3.6, no compilation).  It just pokes along, I've got plenty of memory and my cpu's sitting around 0.5 (all procs).  Probably net speedup from 4 cores is 1.5x a one core system.  Not touching the disk.

I have to wonder if these were nd.array if it wouldn't go a lot faster. Since everthing(?) is integer based, I don't see why I couldn't swap out those lists for arrays.

Damn, got all the way through and then bombed pickling the weights.  I fixed the bug (file needed to be opened 'wb', not 'w') and restarted, this time with the params he suggests.  It takes 100mins to run.

Total elapsed time, 94 min.

Another thing I could do it to run this out of a notebook.  There's no reason that shouldn't work.  

##### Note: I want to rerun best model without Text_3  and Sub_Object_Description.  That may be enough to get me a little higher on LB.

####  mon02jul 

#### back to the game plan

Next thing is to finish milestone report.  Repeating the instructions:

1. Define the problem
1. Identify your client
1. Describe your data set, and how you cleaned/wrangled it
1. List other potential data sets you could use
1. Explain your initial findings
1. Share the Capstone Project 1 code and milestone report related to Github repository

Items 1, 2, 3 are written.  Item 4 is irrelevant.  Item 5 is the key.  What are the initial findings?

1. The EDA is done - maybe clean up for public consumption.  Cleaned up some.
1. Build all of the presented models - done
1. Evaluate all those models with log loss (locally on train/test and via submission) and standard metrics; that can be the initial findings.
>   I don't know that I've done this with all of them. The relevant notebooks are first_models and first_models_metrics.  Time to review them.


* first_models - I've cleaned this and running it now.  Should take about an hour.  235 now, sb done by 330.  Hope it doesn't blow up.  It finished but I noticed a few organization things:

    1. Models are not consistently named and neither are output files.  
    2. Made an error in writing one of the prediction files.  Fixed it.
    3. Renamed (and documented) output .csv files and moved to submitted directory.  I made sure that the outputs were from the correct models.  
    4. Resubmitted Random Forest model and got a different score.  I remember having had some trouble with that submission.  New submission scores better than whatever it was I submitted before.


* first_models_metrics - could stand cleaning, but it's all built and outputs are written and documented.

* new notebook is needed - I need a notebook to apply a suite of metrics to each set of outputs.  I want to see the following on each model:

        * dd_mmll
        * my version of mmll that uses sklearn.metrics.log_loss
        * accuracy score
        * F1 score
        * Classification report
        * Confusion matrices would be nice but it gets hard with big cms
        * ROC AUC??
        
If I make a function to do one, I'll be able to do them all in a loop.  This __*is*__ the findings so it has to be done for the milestone report.

Some (most) of the work I did on metrics is in standard_metrics.ipynb.  

This file does the groundwork but not presentation quality.  What should it do?

* for all models (test only is good enough):
    1. accuracy for all models (test only is good enough)
    2. F1 score
    3. ROC_AUC?  Can this be aggregated


* for some model (maybe mod03, bigrams/Count_Vectorizer, or mod02, just CountVectorizer) demo above shopping list


##### Note: the RandomForestClf needs to have random_state set if you want repeatable results.  I didn't do that and that explains why I've seen different results.

##### Note: need to update file list above.

#### Tuesday 2018jul03

Was working in standard_metrics and I realized that I need explanations about how OneVsRest() predicts outputs and probabilties. 
The work underpinning the explanations is in flat_to_labels and mod0_multiple_ways_v2.

I did the explanation in flat_to_labels and also cleaned up both notebooks.  They're good to go.

Now I can reference that in standard_metrics and proceed.

##### Note: the way I did flat_to_models notebook and python file was maybe a good model for  working - develop in a notebook and then build a python file (the goal here was to build a function).  For output, maybe another notebook can become main and drive fns from .py file.



#### Wed 04jul2018

Still working on standard metrics...  

There's an issue with metrics that I was not quite understanding:

>    Some metrics (accuracy/F1...) use predictions (i.e. labels) and some (log loss, ROC_AUC) use probabilities (floats between 0 and 1).  The prediction-based metrics want y and predictions in hierarchial format.  The probability ones need the binary format, but it needs to be normalized over each target.  
>    I don't have a function to do that.  It would be very similar to flat_to_labels and should probably live in the same place.
    Actually, I remember that I worked on this in mod0_multiple_ways...  

I had a terrible problem with the function seemingly misbehaving.  Care needs to be taken, so as not to smash the input.

Another question is should I just normalize when I make them.  The only question is: will I ever want to use the raw outputs from OneVsRest ever again?  It doesn't likely, but definitely not impossible.

#### thu 05jul2018

Whew.  Standard_Metrics notebook is done.  Where am I at?

for milestone:
* eda - checked it over.
* first_models - checked it
* first_models_metrics - check it 
* standard_metrics - double check it after the two above.

Need to compile all train/test log_loss (mine and theirs), accuracy and F1 plus competition score.
* have DD scores
* need my scores

Need to describe delta between test metrics and competition metric.

Need to describe next steps.

    * deconstruct and go piece-by-piece - what's a win; what's not?
    * pull best pieces, build competitive model, submit, get 5th or 6th and describe.
    
    


#### fri 06jul2018

deeplearning.ai wants me to take a test.  Yikes.

Quicky smash together the Milestone.  Couple of pieces left to put in (numbers, a chart)....  Ship to AJ...

#### sat 07jul 

AJ sent me edits (useful).  He's okay with this for Milestone report and go forward plan.

Need to check for updates an reboot.  Windows seems kind of thrashed (odd little stuff not working, like Word).  

Also need clean machine for test (no problems when I'm on the clock please).  Clean up and do that now.



#### saturday to thursday

* update - no boot
* usual fiddles no go
* open machine: reseat mem, hdd cable
* restart okay, power down
* attach cover, restart: no go
* open and repeat: oops busted connector - machine now screwed

research alternatives

Sunday: buy replacement

Monday: finish up milestone

Tuesday: tinker with rf

Wed: machine comes
Can't swap in my drive because of cable mismatch.  Configure





#### thu 12jul

##### Need to stop obsessing about the busted hardware and get on with it.

* first task - make sure git repo is working 

git status

 modified:   EDA_ddbpfe.ipynb
        modified:   Project_Log.ipynb
        modified:   first_models.ipynb
        modified:   first_models_metrics.ipynb
        modified:   flat_to_labels.ipynb
        modified:   mod0_multiple_ways_v2.ipynb
        deleted:    python/dd_mmll.py~
        deleted:    python/flat_to_labels.py~
        deleted:    python/log_loss.py
        deleted:    python/ms_mmdf.py
        deleted:    python/plot_confusion_matrix.py~
        modified:   standard_metrics.ipynb

git rm python/dd_mmll.py~ python/flat_to_labels.py~ python/log_loss.py python/ms_mmdf.py
git add Project_Log.ipynb first_models.ipynb first_models_metrics.ipynb flat_to_labels.ipynb 
git add mod0_multiple_ways_v2.ipynb standard_metrics.ipynb

##### Big fright over github over.
It looked bad, but it wasn't too hard to recover (non-intuitive though).

Started making sure the right stuff is there.
ok: first_models.ipynb
ok:   first_models_metrics.ipynb
ok:   flat_to_labels.ipynb
ok:   renamed mod0_multiple_ways_v2.ipynb mod0_multiple_ways.ipynb
ok:  standard_metrics.ipynb

All good with these files...

#### Mon 16jul

need to pop up a level.  I'm spinning my wheels on this stupid n_jobs=-1 bug.  I need a plan to get to the end of the project:

1. finish the analysis of the one change at a time approach
2. come up with my best shot - I'd like to get to 4th place with my own approach

I think I should just abandon n_jobs=-1; yes, it slows things down, but it won't break.  Where to from here?
Best model was no numbers, use bigrams in count vectorize (0.67LB).  Without bigrams we get 0.69LB.

I did train a model with the whole dataset and used it to predict.  It did slightly worse.

Another thought is to use the feature interaction code without bigrams. That gives us all the bigrams.  It won't get combinations of bigrams and unigrams directly, but may get some of that indirectly.  Didn't work

Running an experiment to see if the feature interaction model (without numbers) and more features (1600) will do anything.  It's way slow...

Python is slower on linux (at least from running a few fits and predictions).  It seems like it's maybe 20% slower.  It's the same machine, just running ubuntu.  I think this is due to memory - it's the lenovo 16G module.  Machine is at 100%.  It may also be a slower part (my memory is faster than the CPU).

Other things that could help:
* random forest - run model without numbers
* random forest - get a good one then combine with best log reg

Started a run of dd feature interaction model but using 1600 features instead of 300.  Took almost 12hours to run (overnight).  Score 0.7831.  The 300 feature version scored 0.8893.  That one took about an hour.  So, this is not a very effective approach.


#### Tue 17jul

Did a bunch of cranking on random forest models.  The notion is if RF gets the same data I used for my best logistic regression, maybe it will do better.


#### Wed 18jul

Continued with RF.  Good results but not as good as best logistic regression.  Working on tuning it, but I think to do better, we need some feature interaction.  

What I'd like to do is pull out the 100 most important features (as determined by random forest) and compute the cartesian product, then add those to the features.  It's only another 10k features.  That's only a 1/3 of the features we have already.

The difference from what the tutorial does is that he throws away all but the 300 best features and uses their product.  Wait, after looking at that code, what I said is not exactly right; my assumptions there was wrong.  It looks to me like he's doing exactly what I was talking about.

No, what he does is throw away all but the best features.  Then he computes the interaction like I've been talking about and then appends the 300 (or however many he selects) best features with their interactions (90,000) and uses that.

My experiment mod04_99 used 1600 features.  It is still not as good as the raw vectorization (with bigrams).  And it took 12+hours.  The reason it's so slow is that he's using loops to do the multiplication.

Another experiment showed that using the best 15000 features (no interaction) was not as good as using all the features (31k), so all of the original features are to some extent useful.

This could work with either logistic regression or RF.

Decoded what almost all the files are.  Annotated some above.

Tomorrow:
1) Finish up with all open files.  Halt and close.  Reboot.  
2) Tune random forest
    * we need to go all the way down
    * can add some trees (but lots won't help)
    * could look at n_features - it's fairly close to optimal at sqrt
3) How to smash cartesian product of 100 best features onto the end of the features?
    * function transformer?
        It could 
            * do the vectorization
            * do the selection (one RF fit isn't too bad)
            * do the cross product and flatten
            * hstack onto original vectorization

That would be fairly easy to test on simple corpus (maybe just grab a couple of rows out of the data)...
         
This is tricky to figure out in a pipeline because first I need to do the hash vec and then fit an RF,get the indices 100 best from feature_importances, and pull out the values from transformed input.  Then multiply and make sure it's the right shape.
So I'd need a new transformer to do that step:
* input: (n_samples, 100)
* output: (n_samples, 10000)     
Then tack it on. And make sure it all does what I think.
         
issues:
1. making sure I can pull the features by their indices and have that work
2. new transformer (that shouldn't be too bad, though maybe some tricky dimensioning)...
         
Decision: I think I should maybe come back to this after more organization and some writing.

![pipelinefigure](./images/FeatureEnhancement.png)


#### Thu 19jul

Cleaning up.  Think I need a reboot.  I need to look at n_jobs=-1 on linux.  If behavior is the same, I don't need to look at system mechanic interaction on windows.  

Also, if it works, I could run my models there if I could figure out smb.

Seems like using hashing vectorizer instead of count vectorizer helps with n_jobs=-1.  Also, it's no worse than CV, performance wise.  

Quick thought on feature interaction: instead of building my own, I can use his but change the very last step where he slips in the original features.

That might be the way to go.  It should be easier than rolling my own and could see if it helps.  I think it would be reasonable to double the feature size.  Right now, I've got 30k features.  sqrt(30k) is 170, so I could add that many.  His code takes an hour to run with 300 + 300^2/2 features.  I wonder how much time is used computing those 45k features.

One last thought:
1. Could drop out one feature at a time overnight and look at loss (or gain)
2. Could pick a new random seed and rerun to see if I happened to luck out with the split I've got.




##### One of the labels (or features) is supposed to turn all the other ones off.  I should check if my predictions do that.

#### Fri 20jul

Completed manifest.  It's helpful to have a scorecard so you can tell who the players are.

__Quick thought on feature interaction: instead of building my own, I can use his but change the very last step where he slips in the original features.__

From yesterday: to see if this notion helps without rolling my own, I'm going to rework PJ Bull code to do what I want.  I know/trust that it works correctly (though slow) - so I won't have to work out the vectorization and matrix type stuff.

That assumption turned out to be wrong.  I did need to get in and look at every line of that code in order to figure out the right way to do the simple thing I needed to do.  But it does work now..


#### Sat 21jul

am: Continued with JustInteractions transformer.  Fit the first model.  Not in time for result for AJ call.

2p AJ call.

The plan is to finish this week.

Logistics: the whole drive directory is under 6 GB so I can put on a flash and move to linux box.

There's a tiny bit of improvement with interacted best features.  Best setting is 200 best.

What do I do with this clf?
1. pull into separate file
2. fit against entire dataset
3. predict on holdout and build submission file
4. submit

There are another 2 things to check.
1. does selectfrommodel(rf) do better
2. there's a flag in the data that says to fill all fields with NOLABEL.  I could/should do a postproc to make sure that's not hurting me.


>Note, there is a hierarchical relationship for these labels. If a line is marked as Non-Operating in the Operating_Status category, then all of the other labels should be marked as NO_LABEL since ERS does not analyze and compare non-operating budget items.




#### Sun 22jul

It looks like Operating_Status controlling other labels (if set to 'Non Operating') is not causing big scoring losses. 
We don't get that as input because it's a label.  Right now I'm predicting the label correctly like 98.5 percent of the time and it looks like when it's set to non_op, the other labels are doing the right thing.  

Next do the RF selection and see if it's better.  Doubt it but it's a cheap change (to code).  Well, pretty cheap.  Had to figure out how important the 200th most important feature is to use that as a threshold to SelectFromModel.

In process.  Taking a long time.  200 features from SelectKBest took 100 minutes.  Seems like it ought to be done.  Not sure why but it took much longer than the SelectKBest version.

Result essentially the same.  Comp score was not better.

In [4]:
import time; time.localtime()

time.struct_time(tm_year=2018, tm_mon=7, tm_mday=22, tm_hour=22, tm_min=11, tm_sec=33, tm_wday=6, tm_yday=203, tm_isdst=1)

#### Sun cont.

I need to chart the metrics for all the original models including the submitted scores.  Those should be in standard_metrics.ipynb.  Contest scores should be in first_models...  

Okay, I have the agg. log loss, accuracy and F1 for all the tutorials in scoring.xls

What else do I need?  All the scores from all my incremental models.  Those are in one_at_a_time_part1/part3.

* part 1

Don't have all the scores; should revise to get them.  Nothing in here takes very long.

* part 3



#### Well.  My interactions didn't help though I fit the test data better.  I'm starting to conclude that I have a bit of overfitting and that's why even though things look great on my end, I get worse numbers on the holdout.

One thing I haven't tried even though it's reasonably easy is to ensemble a good RF model with best log reg.  That's not hard compared to other things I've done.  Refitting the best RF that's handy.  Going to predict, then read both, average and write out.

That improved things.  Can check 1/3RF, 2/3LR and the reverse when I get more submissions.

#### mod_best_n_jobs_plus file needs to reorganized - too much is going on in there...

#### Mon 23jul

Can't submit more because yesterday's submissions were actually today :-(.

Made a new file called ensemble.ipynb.  Combines submission files.  The first one I did was an RF with strongest LR.  Contest score was better than log reg alone.  The RF (with local log loss 0.15) scored over 2 (undoubtedly because of overconfident wrong answers).

Also, another new file with regularization. Using 1/10 of the data, I looked at values of C.  Trying one now on the whole dataset with C=0.0125.  I think that should put me at about 0.3 log loss.  Here are the results: log loss on training set: 0.1386; log loss on test set: 0.1412.  Looks pretty good,

Can't resub because I used up all my chances for today at 1 am....

__Nailed it__

So the first isn't overfit.  The RF is.  The 3rd model isn't overfit.

#### Regularization is key - Mod3_1 __*was*__ overfit.  With regularization (C=0.1) scored 0.5341 (0.6599 without).  There may be a bit more.

Also Mod_400 scored high with some regularization.  There's more available

#### Tuesday

#### Wednesday

1020a out of submissions for today.  Dropping Mod_reg to 0.1 improved over 0.01 (.5341 vs .5400).   I doubt there's much more available.  Next one should be 0.33 - that'll tell me if I overshot the sweet spot. 

Wrote big chunks of report.

#### thu 26jul2018

Just finally finished readme file list.  Whew.

What's left: I need to do two things

* regularize things that make sense to regularize
    1. mod4 - I don't know if this is good enough but it's worth trying
    2. mod3_best - I have some numbers but I want to bracket minimum log loss and plot
    3. mod_400 - same
    
    Made some mistakes last night and I'm confused about what model scored what.  I need to get that straighted out.
    
    I need plots for all 3 of those things with at least 5 points for C.
    
* Write all of that up.

Oh, yeah: I also need an appendix to talk about multi-target-multi-class and what it means.
    
    
    
Okay, I've got to be a bit more scrupulous about this deal here.  I noticed that mod3_1_reg_third has a scaler.  Mod3_1_reg does not.  mod_reg_01 does not.  mod_reg_tenth.  Those are the models in mod_3_1_reg.

Also, I have another data point for this clf - C=0, i.e. original run...  I did that with and without scaling (was worse with).

* Mod3
These are more or less straightened out.  I'd like one more point for my graph, C=0.00333.

* mod_400_reg
I have one for C=1/16
I hoped to get another data point for C=1/8; goofed. edited everything but C.  Run before bed.

Tomorrow, I can build another prediction on the other side in the am.


mod4
I've only got the unreged version.  Could build run these all on linux box tomorrow.
1/2 __1/4__ 1/8 __/16__ 1/32 __1/64__ 1/128 __1/256__ 1/512 __1/1028__ or

1/3 1/10  1/30 1/100 1/300 1/1000






#### fri 27jul2018

Goofed up again on submission.  

But now I'm going to do some info gathering before submitting.  Get the log loss for train and test into excel.  That way, I'll see if I've built the wrong thing.

Got a good idea for my example.  See mcml.ipynb.




#### sat/sun/mon

* sat

Submitted 3 more predictions for mod_400_reg, 1/32, 1/64, 1/128.  Didn't get the improvement I was hoping for.  

I don't understand why I have such great log loss and it doesn't score that well.  

Things I could do:

* put Total back in (abs value)  did this.  not great.  would be better with scaling
    * 0.6237 mod_total_reg_033;  include Total; no scaling
    * 0.5999 mod_total_reg_033_scale ;  include Total; scaling
    Worse with Total included as a numerical column.  Could bin into 20% bins (5) and turn into categorical.  One thing that might be helpful is that whatever way you handle total, it shouldn't hurt log loss without regularization.  Then you could know that you could get better by including it.
    

* could fiddle with mod_200 - didn't explore regularization adequately; also, fit is fastish (15min).
    
    * don't know what regularization was on the one I submitted: 
        * -rwxrwxrwx 1 saus saus 106961298 Jul 23 22:17 pred_mod_200.csv;
        * 07/30/2018  03:24 PM       106,426,437 pred_mod_200_0009.csv compares different
        I think it might have been 0.0009.  Checking now: rebuild with C=0.0009, compare to submission/pred_mod_200.csv. They are not the same; it may be because the seed changed.
        * 07/30/2018  04:12 PM       107,793,536 pred_mod_200_003.csv  compares different
        Blurb at top says 0.003 with score.  Checking now: rebuild with C=0.003, compare to submission/pred_mod_200.csv. Bottom line: I don't really know what the 23jul one is.

* CountVectorizer 
    ignore things that happen only once... 
    * fiddled with this a bit - some reduction in token count
    


### Important
* __train on whole dataset (that takes the split out of the game, also more chance to train on rare labels, features)__


* reduce features before classification
    * didn't make a diff using 10k before clf, but not bad, score 0.5819
    * may be worth looking at a bit less lambda...


* reduce features out of the vectorizer - we know 300 isn't good enough.  Didn't we already do this for mod4?  yes, tried 1600 features with interaction, that wasn't bad but took forever.

    * tried this a different way: in mod_400, after computing the interactions and joining original features, reduced to 10k best features - made essentially no difference.  see above....
    

* __maybe the place to hack around is mod3_1 with regularization C=0.033 - that's a 5 min fit.__
    * did this with Total, still need to try scaling with that one
    * did that: 0.5999


* I never scored mod_200 and mod_400 without regularization
    * mod_200 no reg: 0.7263
    * mod_400 no reg: 

### I've got too many levers and arbitrary evaluation.  That's an untenable situation for progress.

### I need to finish.  What's the driver(s) for my obsession?

1. Why is the local score different from the comp score?  this is the question that's driving me nuts.
2. Would like to be in 1st place because that would justify all this flailing
    * Yes, it's like gambling - the big payout will make everything ok.
3. At this point I'm convinced that the holdout is a different distribution.



#### What have I learned:

* there's got to be a better way to organize models/predictions with slightly different characteristics than changing the name of the notebook/output file.
* it would help to be able to pickle the models after fitting
* broader knowledge of libraries would help - there's lots of help out there, if you knew about it.
    * couldn't figure out FeatureHasher
    * didn't know about difflib, etc.

#### Tue 31jul2018

* What's in play?

    * mod_400_noreg - cooking now - submit for 2nd example of just_interactions - forgot to fix C - again.
    * mod3_1_reg_033_final (trained on full dataset) needs a submission: 0.5228, a smidge better - actually it's the same.
    * Do I need a concluding remarks - oh yeah, it should be "future work".  That would allow me to couch my commentary in a way that's not critical of DrivenData.
    * appendix on multi-class, multi-class...



#### wed 01aug2018

Tried to run mod_1000 last night, but forgot to select 1000 features.  Moved it to linux; running now.  Probably take all day.
That blew up.  Not enough memory.

Lots of work on appendix.

#### thu 02aug

1k features has 0.04 log loss on test set.  DD score 0.7462.  Regularizing would help, but how much?  400 features noreg was 0.7307, best was 0.5818 at 1/32.  I'm guessing 0.01 is probably about right for this one because it has more variance.  Can check overnight.

Finished appendix early.  Edited later.  Semi-problematic stitching it into report.

It's done.  Off to AJ.

Here's my assessment: the DD methodology is bound to fall short of quocle approach because he used a better feature set:  positioned(?) full text features, words, positioned words, binned and categorized numerical features, hand-selected interactions of full-text features (2s and 3s).  The holdout distribution is different from available data, so regularization is necessary but can't be done against available data.  Believe tinrtgu clf does early stopping and thus auto-regularizes.



Email Chain
```
A J Sanchez
6:31 AM (12 hours ago)

Hi Mark,

Thank you for the reminder!

I would add a section explicitly called “Recommendations for the Clients”—to comply with the Capstone Project Guidelines document.

A related discussion, as part of your analysis of results, would be addressing “feature importance”, per class/label.

Regards,
             -AJ
===
Mark Sausville <saus52@gmail.com>
10:47 AM (8 hours ago)

AJ, I can do the first suggestion.  

As to the second, I have 31k features after pre-processing.  I'm not sure how to address the feature importance in relation to class/label pair that will be useful.  I have the feature importances laying around somewhere but they relate to all 104 classifications and aren't differentiated.

Another issue that I don't have the labels for the holdout set.  Because of that I can't  tell where my misclassifications are.  If I had the labels, I could do the error analysis and show error rates for class/labels pairs, which could be useful.

Maybe I should just show how that's done for labeled test set and then say the above.

Or am I misunderstanding your comment.
===
A J Sanchez
11:06 AM (7 hours ago)
to saus

Hey Mark,

OK about the first suggestion.

About the second suggestion ... could you use the Gini scores from the Random Forest model and comment on the top 10? I understand the interpretability issue you mention ... so if you think it cannot be done in a practical way, mention it and explain why ...

Regards,
             -AJ
```

#### tue 07aug


Need to make some edits:

What would I tell a client (ERS looking to partially automate labeling budgets).

The best scoring classifier we produced has worst case accuracy (and F1 score) of 95% on the test set.  It's not possible to say with certainty what accuracy will be on similar documents but given the difference in log loss between the test and holdout sets, it is likely that accuracy will still be very good.  Our experience with a range of models tells us that better than 90% accuracy/F1 can be expected from models with log loss of 0.52 (our best model's score on the holdout set).  

Across all the models, the lowest accuracy was on the target, 'Function'.  This target has 37 possible labels, some of which are very rare.  Human analysts should pay close attention to this target to validate the model's choice of label.  

For a better idea of where human analysts should also be alert, the confusion matrices for each from the classification could be  examined to get a better idea which labels are most likely to be labeled incorrectly.

Feature Importances:

Blah, blah.....

Did it


#### wed 08aug

AJ says ship it.

2 things left
* powerpoint
* clean and update github

What goes into the PPT???

It's done - basically summary of report.





# Clean up!